<a href="https://colab.research.google.com/github/plaban1981/Agents/blob/main/Crewai_Flows_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Required Dependencies

In [ ]:
!pip install crewai[tools]==0.86.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 10.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of crewai-tools to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.3 MB/s eta 0

In [ ]:
!pip install tqdm==4.67.1


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:

!pip install firecrawl-py


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
%pip install crewai

  Using cached crewai-0.98.0-py3-none-any.whl.metadata (27 kB)
  Using cached auth0_python-4.7.2-py3-none-any.whl.metadata (8.9 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached instructor-1.7.2-py3-none-any.whl.metadata (18 kB)
  Using cached litellm-1.57.4-py3-none-any.whl.metadata (36 kB)
  Using cached openai-1.60.1-py3-none-any.whl.metadata (27 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_api-1.29.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.29.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_sdk-1.29.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached pdfplumber-0.11.5-py3-none-any.whl.metadata (42 kB)
  Using cached pyvis-0.3.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached aiohttp-3.11.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.

## Setup Required APIs

In [ ]:

import os
os.environ['FIRECRAWL_API_KEY'] = "fc-f2a0d481accf4247974f82199e8b8f08"
os.environ['TYPEFULLY_API_KEY'] = "jigPDSPD0F6x1CCT"


## Typefully Scheduler

In [ ]:
import os
import json
import requests
import datetime
from typing import Optional, Dict, Any
from pydantic import BaseModel

# Constants
API_URL = "https://api.typefully.com/v1/drafts/"
API_KEY = "jigPDSPD0F6x1CCT"
HEADERS = {
    "X-API-KEY": f"Bearer {API_KEY}"
}

def json_to_typefully_content(thread_json: Dict[str, Any]) -> str:
    """Convert JSON thread format to Typefully's format with 4 newlines between tweets."""
    tweets = thread_json['tweets']
    formatted_tweets = []
    for tweet in tweets:
        tweet_text = tweet['content']
        if 'media_urls' in tweet and tweet['media_urls']:
            tweet_text += f"\n{tweet['media_urls'][0]}"
        formatted_tweets.append(tweet_text)

    return '\n\n\n\n'.join(formatted_tweets)

def json_to_linkedin_content(thread_json: Dict[str, Any]) -> str:
    """Convert JSON thread format to Typefully's format."""
    content = thread_json['content']
    if 'url' in thread_json and thread_json['url']:
        content += f"\n{thread_json['url']}"
    return content

def schedule_thread(
    content: str,
    schedule_date: str = "next-free-slot",
    threadify: bool = False,
    share: bool = False,
    auto_retweet_enabled: bool = False,
    auto_plug_enabled: bool = False
) -> Optional[Dict[str, Any]]:
    """Schedule a thread on Typefully."""
    payload = {
        "content": content,
        "schedule-date": schedule_date,
        "threadify": threadify,
        "share": share,
        "auto_retweet_enabled": auto_retweet_enabled,
        "auto_plug_enabled": auto_plug_enabled
    }

    payload = {key: value for key, value in payload.items() if value is not None}

    try:
        response = requests.post(API_URL, json=payload, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

def schedule(
    thread_model: BaseModel,
    hours_from_now: int = 1,
    threadify: bool = False,
    share: bool = True,
    post_type: str = "twitter"
) -> Optional[Dict[str, Any]]:
    """
    Schedule a thread from a Pydantic model.

    Args:
        thread_model: Pydantic model containing thread data
        hours_from_now: Hours from now to schedule the thread (default: 1)
        threadify: Whether to let Typefully split the content (default: False)
        share: Whether to get a share URL in response (default: True)

    Returns:
        API response dictionary or None if failed
    """
    try:
        # Convert Pydantic model to dict
        thread_json = thread_model.pydantic.model_dump()
        print("######## Thread JSON: ", thread_json)
        # Convert to Typefully format
        if post_type == "twitter":
            thread_content = json_to_typefully_content(thread_json)
        elif post_type == "linkedin":
            thread_content = json_to_linkedin_content(thread_json)

        # Calculate schedule time
        schedule_date = (datetime.datetime.utcnow() +
                        datetime.timedelta(hours=hours_from_now)).isoformat() + "Z"

        # Schedule the thread
        response = schedule_thread(
            content=thread_content,
            schedule_date=schedule_date,
            threadify=threadify,
            share=share
        )

        if response:
            print("Thread scheduled successfully!")
            return response
        else:
            print("Failed to schedule the thread.")
            return None

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

## Install Required Libraries

In [ ]:
# Importing necessary libraries
import getpass
import os
import datetime
import uuid
import yaml
import json
import subprocess
from pathlib import Path
import pydantic
from pydantic import BaseModel
from typing import Optional

# Firecrawl SDK
from firecrawl import FirecrawlApp


# Importing Crew related components
from crewai import Agent, Task, Crew, LLM

# Importing CrewAI Flow related components
from crewai.flow.flow import Flow, listen, start, router, or_

from dotenv import load_dotenv
load_dotenv()
# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

## Using Ollama

## Set up LLM

In [ ]:
from crewai import LLM

import litellm
#litellm.request_timeout=3600
#llm = LLM(model="groq/llama-3.3-70b-versatile")
#llm = LLM(model="openai/gpt-4o-mini")
#llm = LLM("ollama/llama3.2",timeout=3600)
llm = LLM("ollama/deepseek-v2")

In [ ]:
blog_post_url = "https://blog.dailydoseofds.com/p/hands-on-multi-agent-youtube-video"

## Twitter Thread Planning Crew

In [ ]:
class Tweet(BaseModel):
    """Represents an individual tweet in a thread"""
    content: str
    is_hook: bool = False  # Identifies if this is the opening/hook tweet
    media_urls: Optional[list[str]] = []  # Optional media attachments (images, code snippets)

class Thread(BaseModel):
    """Represents a Twitter thread"""
    topic: str  # Main topic/subject of the thread
    tweets: list[Tweet]  # List of tweets in the thread

In [ ]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
)

draft_analyzer = Agent(role ="Draft Analyzer",
                       goal="Analyze draft and identify key ideas, sections and technical concepts",
                       backstory=(""" You are a technical writer with years of experience writing, editing and
    reviewing technical blogs. You have a talent for understanding and
    documenting technical concepts."""),
                      tools=[DirectoryReadTool(),FileReadTool()],
                      verbose=True,
                      llm=llm)
#
twitter_thread_planner = Agent(role="Twitter Thread Planner",
                               goal="Create a Twitter thread plan based on the provided draft analysis",
                               backstory=(""" You are a technical writer with years of experience in converting long
    technical blogs into Twitter threads. You have a talent for breaking longform
    content into bite-sized tweets that are engaging and informative. And identify
    relevant urls to media that can be associated with a tweet."""),
                               tools=[DirectoryReadTool(),FileReadTool()],
                               verbose=True,
                               llm=llm)
#
# TASK
#
analyze_draft = Task(
  description=("""Analyze the markdown file at {draft_path} to create a developer-focused
    technical overview

    1. Map out the core idea that the blog discusses

    2. Identify key sections and what each section is about

    3. For each section, extract all URLs that appear inside image markdown syntax ![](image_url)

    4. You must associate these identified image urls to their correspoinding sections, so that we can use them with the tweets as media pieces

    Focus on details that are important for a comprehensive understanding of
    the blog."""),
  expected_output="""|
    A technical analysis containing:
    - Blog title and core concept/idea
    - Key technical sections identified with their main points
    - Important code examples or technical concepts covered
    - Key takeaways for developers
    - Relevant urls to media that are associated with the key sections and can be associated with a tweet, this must be done""",
  agent=draft_analyzer
)
create_twitter_thread_plan = Task(
  description=("""|
    Develop an engaging Twitter thread based on the draft analysis provided and closely follow the writing style prvided in the {path_to_example_threads}

    The thread should break down complex technical concepts into digestible, tweet-sized chunks
    that maintain technical accuracy while being accessible.

    Plan should include:
    - A strong hook tweet that captures attention, it should be under 10 words, it must be same as the title of the blog
    - Logical flow from basic to advanced concepts
    - Code snippets or key technical highlights that fit Twitter's format
    - Relevant urls to media that are associated with the key sections and must be associated with their corresponding tweets
    - Clear takeaways for engineering audience

    Make sure to cover:
    - close follow the writing style provided in the {path_to_example_threads}
    - The core problem being solved
    - Key technical innovations or approaches
    - Interesting implementation details
    - Real-world applications or benefits
    - Call to action for the conclusion
    - Add relevant urls to each tweet that can be associated with a tweet

    Focus on creating a narrative that technical audiences will find valuable
    while keeping each tweet concise, accessible and impactful."""),
  expected_output="""|
    The response should be in a JSON format with 2 keys 'topic' and 'tweets'.
    the 'tweets' key has values as a Twitter thread with a list of tweets, where each tweet has the following:
    - content
    - urls to media that are associated with the tweet, whenever possible
    - is_hook: true if the tweet is a hook tweet, false otherwise
    """,
  agent=twitter_thread_planner,
  output_pydantic=Thread
)

planning_crew = Crew(
    agents=[draft_analyzer, twitter_thread_planner],
    tasks=[analyze_draft, create_twitter_thread_plan],
    verbose=False
)

2025-01-25 17:12:38,355 - 138197270736896 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
!mkdir workdir

In [ ]:
!mkdir thread

## LinkedIn Post Planning Crew

In [ ]:
class LinkedInPost(BaseModel):
    """Represents a LinkedIn post"""
    content: str
    media_url: str # Main image url for the post

In [ ]:
linkedin_post_planner = Agent(role="LinkedIn Post Planner",
                              goal="Create an engaging LinkedIn post based on the provided draft analysis",
                              backstory=("""You are a technical writer with extensive experience crafting technical LinkedIn
    content. You excel at distilling technical concepts into clear, authoritative posts
    that resonate with a professional audience while maintaining technical accuracy.
    You know how to balance technical depth with accessibility and incorporate relevant
    hashtags and mentions to maximize engagement."""),
                              verbose=True,
                              llm=llm)

create_linkedin_post_plan = Task(
  description=("""|
    Develop a comprehensive LinkedIn post based on the draft analysis provided

    The post should present technical content in a professional, long-form format
    while maintaining engagement and readability.

    Plan should include:
    - An attention-grabbing opening statement, it should be same as the title of the blog
    - Well-structured body that breaks down the technical content
    - Professional tone suitable for LinkedIn's business audience
    - One main blog URL placed strategically at the end of the post
    - Strategic use of line breaks and formatting
    - Relevant hashtags (3-5 maximum)

    Make sure to cover:
    - The core technical problem and its business impact
    - Key solutions and technical approaches
    - Real-world applications and benefits
    - Professional insights or lessons learned
    - Clear call to action
    - Use Emojis whenever possible

    Focus on creating content that resonates with both technical professionals
    and business leaders while maintaining technical accuracy."""),
  expected_output="""|
    The response should be in a JSON format with 2 keys "content" and "media_url"
    A LinkedIn post plan containing:
      - content
      - a main blog url that is associated with the post""",
  agent=linkedin_post_planner,
  output_pydantic=LinkedInPost
)

linkedin_planning_crew = Crew(
    agents=[draft_analyzer, linkedin_post_planner],
    tasks=[analyze_draft, create_linkedin_post_plan],
    verbose=False
)

2025-01-25 17:42:05,610 - 138197270736896 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
#blog_post_url = "https://blog.dailydoseofds.com/p/build-human-like-memory-for-your"
blog_post_url = "https://blog.dailydoseofds.com/p/traditional-vs-graph-rag"

## CrewAI Flow

In [ ]:
from crewai.flow.flow import Flow, listen, start, router, or_

class ContentPlanningState(BaseModel):
  """
  State for the content planning flow
  """
  blog_post_url: str = blog_post_url
  draft_path: Path = "workdir/"
  post_type: str = "twitter"
  path_to_example_threads: str = "workdir/example_threads.txt"

class CreateContentPlanningFlow(Flow[ContentPlanningState]):
  # Scrape the blog post
  # No need for AI Agents on this step, so we just use regular Python code
  @start()
  def scrape_blog_post(self):
    print(f"# fetching draft from: {self.state.blog_post_url}")
    app = FirecrawlApp(api_key=os.getenv("FIRECRAWL_API_KEY"))
    scrape_result = app.scrape_url(self.state.blog_post_url, params={'formats': ['markdown', 'html']})
    try:
      title = scrape_result['metadata']['title']
    except Exception as e:
      title = str(uuid.uuid4())
    self.state.draft_path = f'workdir/{title}.md'
    with open(self.state.draft_path, 'w') as f:
      f.write(scrape_result['markdown'])
    return self.state

  @router(scrape_blog_post)
  def select_platform(self):
    if self.state.post_type == "twitter":
      return "twitter"
    elif self.state.post_type == "linkedin":
      return "linkedin"

  @listen("twitter")
  def twitter_draft(self):
    print(f"# Planning content for: {self.state.draft_path}")
    result = planning_crew.kickoff(inputs={'draft_path': self.state.draft_path, 'path_to_example_threads': self.state.path_to_example_threads})
    print(f"# Planned content for {self.state.draft_path}:")
    for tweet in result.pydantic.tweets:
        print(f"    - {tweet.content}")
    return result

  @listen("linkedin")
  def linkedin_draft(self):
    print(f"# Planning content for: {self.state.draft_path}")
    result = linkedin_planning_crew.kickoff(inputs={'draft_path': self.state.draft_path})
    print(f"# Planned content for {self.state.draft_path}:")
    print(f"    - {result.pydantic.content}")
    print(f"Linkedin Draft :{result.raw}")

    return result.raw

  @listen(or_(twitter_draft, linkedin_draft))
  def save_plan(self, plan):
    with open(f'thread/{self.state.draft_path.split("/")[-1]}_{self.state.post_type}.json', 'w') as f:
        json.dump(plan.pydantic.model_dump(), f, indent=2)

  @listen(or_(twitter_draft, linkedin_draft))
  def publish(self, plan):
    print(f"# Publishing thread for: {self.state.draft_path}")
    ## Schedule for 1 hour from now
    response = schedule(
        thread_model=plan,
        post_type=self.state.post_type
    )
    print(f"# Thread scheduled for: {self.state.draft_path}")
    print(f"Here's the link to scheduled draft: {response['share_url']}")

In [ ]:
# Plot the flow
flow = CreateContentPlanningFlow()
flow.plot()





Plot saved as crewai_flow.html


## Run Twitter Content Planner Crew (ollama)

In [ ]:
response = planning_crew.kickoff(inputs={'draft_path': "workspace/workdir", 'path_to_example_threads': "workspace/thread/e_thread.txt"})

# Agent: Draft Analyzer
## Task: Analyze the markdown file at workspace/workdir to create a developer-focused
    technical overview

    1. Map out the core idea that the blog discusses

    2. Identify key sections and what each section is about

    3. For each section, extract all URLs that appear inside image markdown syntax ![](image_url)

    4. You must associate these identified image urls to their correspoinding sections, so that we can use them with the tweets as media pieces

    Focus on details that are important for a comprehensive understanding of
    the blog.


/usr/local/lib/python3.10/dist-packages/crewai/tools/tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"




# Agent: Draft Analyzer
## Thought: Thought: To begin analyzing the markdown file located in workspace/workdir, I need first to list files within this directory to understand the structure of the document. This will help identify any existing headings or sections that might guide my analysis and ensure a comprehensive overview is captured for the final answer.
## Using tool: [List files in directory]
## Tool Input: 
"{\"directory\": \"workspace/workdir\"}"
## Tool Output: 
File paths: 
-


/usr/local/lib/python3.10/dist-packages/crewai/tools/tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"


 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FileReadToolSchema
file_path
  Field required [type=missing, input_value={'file': 'workspace/workdir/_markdown_post.md'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing.
 Tool Read a file's content accepts these inputs: A tool that can be used to read None's content.



# Agent: Draft Analyzer
## Thought: Thought: Having listed the files, I now need to determine which file contains the actual content of interest - the markdown blog post that needs analysis. This will be done by reviewing each file for indications or titles that suggest it pertains to technical topics related to software development and blogging about them. The chosen file should ideally contain a main body with headings as they would outline sections within the document, which is essential for further analysis.
## Using tool: [Read a file's conten

...



# Agent: Draft Analyzer
## Thought: To perform the task of analyzing the markdown file at workspace/workdir to create a developer-focused technical overview as per the given criteria, I would first need access to the content of the markdown file. However, since that is not provided here, I will outline an approach using the tools available (List files in directory and Read a file's content), assuming they are used on the actual markdown document for real-time analysis:
Thought: To begin, I should familiarize myself with all sections of the markdown file to ensure comprehensive understanding. This would involve reading each section and noting key points covered therein.
## Using tool: [Read a file's content]
## Tool Input: 
"{\"file_path\": \"/workspace/workdir/blog-overview.md\"}"
## Tool Output: 
Error: File not found at path: /workspace/workdir/blog-overview.md


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: List f

/usr/local/lib/python3.10/dist-packages/crewai/tools/tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"




# Agent: Draft Analyzer
## Thought: Thought: Without access to specific content of a markdown file, it is challenging to provide an accurate analysis meeting all criteria outlined in the task description. However, assuming there are typical sections such as Introduction, Detailed Analysis, and Conclusion present in similar documents for real-time analysis would be necessary.
## Using tool: [List files in directory]
## Tool Input: 
"{\"directory\": \"/workspace/workdir/\"}"
## Tool Output: 
File paths: 
-/workspace/workdir/Evaluation and Observability for LLM Apps - by Avi Chawla.md
 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action:' after 'Thought:'. I will do right next, and don't use a tool I have already used.

If you don't need to use any more tools, you must give your best complete final answer, make sure it satisfy the expect criteria, use the EXACT format below:

Thought: I now can give a great answer
Final Answer: my best comple

/usr/local/lib/python3.10/dist-packages/crewai/tools/tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"


 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FileReadToolSchema
file_path
  Field required [type=missing, input_value={'file': {'path': '/path/to/draft_analysis.txt'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing.
 Tool Read a file's content accepts these inputs: A tool that can be used to read None's content.



# Agent: Twitter Thread Planner
## Thought: Thought: Before proceeding with creating a Twitter thread plan based on the provided draft analysis, I need to ensure that my understanding of the task and requirements is clear. The objective is to create engaging Twitter threads from complex technical content, breaking down concepts into bite-sized tweets while maintaining accuracy and accessibility. This requires careful planning and consideration of how best to structure the information for maximum impact within the character limit.
## Using tool: R

## Run Twitter Content Planner Crew (gpt-4o-mini)

In [ ]:
response = planning_crew.kickoff(inputs={'draft_path': "workdir", 'path_to_example_threads': "/content/thread/e_thread.txt"})

# Agent: Draft Analyzer
## Task: Analyze the markdown file at workdir to create a developer-focused
    technical overview

    1. Map out the core idea that the blog discusses

    2. Identify key sections and what each section is about

    3. For each section, extract all URLs that appear inside image markdown syntax ![](image_url)

    4. You must associate these identified image urls to their correspoinding sections, so that we can use them with the tweets as media pieces

    Focus on details that are important for a comprehensive understanding of
    the blog.


# Agent: Draft Analyzer
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"workdir\"}"
## Tool Output: 
File paths: 
-workdir/Evaluation and Observability for LLM Apps - by Avi Chawla.md
- workdir/Build Human-like Memory for Your AI Agents - by Avi Chawla.md


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: List files in directory
To

In [ ]:
print(response.raw)

{
  "topic": "Evaluation and Observability for LLM Apps",
  "tweets": [
    {
      "content": "Evaluation and Observability for LLM Apps",
      "is_hook": true,
      "media_urls": []
    },
    {
      "content": "Monitoring LLM applications is crucial! Effective evaluation can significantly boost performance and reliability. 🚀",
      "is_hook": false,
      "media_urls": []
    },
    {
      "content": "Introducing Opik: an open-source platform designed to help developers evaluate and observe their LLMs seamlessly! 🌐",
      "is_hook": false,
      "media_urls": ["https://github.com/comet-ml/opik"]
    },
    {
      "content": "Key Features of Opik:\n1️⃣ Record LLM response generation.\n2️⃣ Compare multiple LLM outputs.\n3️⃣ Log traces in production.\n4️⃣ Detect hallucinations with built-in judges. 🔍",
      "is_hook": false,
      "media_urls": []
    },
    {
      "content": "Opik simplifies the debugging process with real-time monitoring - a game changer for LLM development!

#### Ollama Deepsekv2

In [ ]:
response = linkedin_planning_crew.kickoff(inputs={'draft_path':"workdir"})

# Agent: Draft Analyzer
## Task: Analyze the markdown file at workdir to create a developer-focused
    technical overview

    1. Map out the core idea that the blog discusses

    2. Identify key sections and what each section is about

    3. For each section, extract all URLs that appear inside image markdown syntax ![](image_url)

    4. You must associate these identified image urls to their correspoinding sections, so that we can use them with the tweets as media pieces

    Focus on details that are important for a comprehensive understanding of
    the blog.


/usr/local/lib/python3.10/dist-packages/crewai/tools/tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"




# Agent: Draft Analyzer
## Thought: Thought: To begin analyzing the markdown file at workdir for creating a developer-focused technical overview, I need to first identify the core idea discussed in the blog. This will help me understand the overall context of the content and guide my analysis process.
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"<Path_to_workdir>\"}"
## Tool Output: 
File paths: 
-


/usr/local/lib/python3.10/dist-packages/crewai/tools/tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"




# Agent: Draft Analyzer
## Thought: Thought: After listing the files in the specified directory, I now have a clearer picture of all content available for analysis. This will enable me to identify key sections and associate them with their corresponding image URLs. It's crucial to understand how these media elements support and enhance each section's main points.
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"<File_path_to_markdown_file>\"}"
## Tool Output: 
Error: File not found at path: <File_path_to_markdown_file>


# Agent: Draft Analyzer
## Thought: Thought: It appears there was an error in accessing the markdown file for content extraction, which is necessary to proceed with my analysis tasks as outlined by the user. This technical hiccup requires immediate attention to ensure that all subsequent steps can be effectively executed without hindrance.
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"<Path_to_workdir>\"}"
## Tool Ou

In [ ]:
print(response.raw)

Given the lack of specific detail from the draft analysis, here’s a hypothetical approach to crafting an engaging LinkedIn post focusing on AI and its impact in the business sector:

{
  "content": "The integration of Artificial Intelligence (AI) into business operations is not just about enhancing efficiency; it's about revolutionizing how we perceive data. As businesses continue to harness AI, they are unlocking insights that were once unimaginable.\n\nFrom predictive analytics helping in inventory management and sales forecasting to advanced chatbots providing customer support 24/7, the benefits of AI span across every aspect of business processes—improving efficiency and reducing costs while enhancing customer satisfaction. This is a paradigm shift where tech-savvy businesses are leveraging data insights to stay ahead in an increasingly competitive market.\n\nThe core technical problem lies in processing vast amounts of structured and unstructured data, which was traditionally too 

## Run LinkedIN Content planner Crew - gpt4o-mini

In [ ]:
linkedin_planning_crew.kickoff(inputs={'draft_path':"workdir"})

# Agent: Draft Analyzer
## Task: Analyze the markdown file at workdir to create a developer-focused
    technical overview

    1. Map out the core idea that the blog discusses

    2. Identify key sections and what each section is about

    3. For each section, extract all URLs that appear inside image markdown syntax ![](image_url)

    4. You must associate these identified image urls to their correspoinding sections, so that we can use them with the tweets as media pieces

    Focus on details that are important for a comprehensive understanding of
    the blog.


# Agent: Draft Analyzer
## Thought: I need to access the contents of the markdown file in the work directory to begin the analysis.
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"workdir\"}"
## Tool Output: 
File paths: 
-workdir/Evaluation and Observability for LLM Apps - by Avi Chawla.md
- workdir/Build Human-like Memory for Your AI Agents - by Avi Chawla.md


You ONLY have access to the foll

CrewOutput(raw='{\n  "content": "🚀 **Evaluation and Observability for LLM Apps** 🚀\\n\\nAs the integration of Large Language Models (LLMs) becomes increasingly essential across industries, the need for robust monitoring and debugging mechanisms is more pressing than ever. Understanding how to evaluate the performance of LLM applications not only safeguards against errors but also aligns with business objectives and enhances user experience.\\n\\n### **Why is Monitoring LLMs Crucial?**\\nLLMs are complex systems that generate responses based on vast amounts of data. Therefore, without effective monitoring, we risk inconsistent outputs that can directly impact business operations and user trust. Integrating evaluation and observability into LLMs allows developers to ensure quality, reliability, and accountability.\\n\\n### **Introducing Opik: The Game-Changer**\\nWe can leverage **Opik**, an open-source platform designed specifically to enhance evaluation and observability for LLM applic

In [ ]:
from IPython.display  import display,HTML,Image
display(HTML("/workspace/crewai_flow.html"))


In [ ]:
post_type = "linkedin"
#blog_post_url = "https://blog.dailydoseofds.com/p/5-agentic-ai-design-patterns"
blog_post_url = "https://blog.dailydoseofds.com/p/evaluation-and-observability-for"
flow = CreateContentPlanningFlow()
flow.state.post_type = post_type
flow.state

ContentPlanningState(blog_post_url='https://blog.dailydoseofds.com/p/traditional-vs-graph-rag', draft_path='workdir/', post_type='linkedin', path_to_example_threads='workdir/example_threads.txt')

#### Ollama Llama3.2

In [ ]:
response = flow.kickoff()

# fetching draft from: https://blog.dailydoseofds.com/p/traditional-vs-graph-rag
# Planning content for: workdir/Traditional vs. Graph RAG .md
# Agent: Draft Analyzer
## Task: Analyze the markdown file at workdir/Traditional vs. Graph RAG .md to create a developer-focused
    technical overview

    1. Map out the core idea that the blog discusses

    2. Identify key sections and what each section is about

    3. For each section, extract all URLs that appear inside image markdown syntax ![](image_url)

    4. You must associate these identified image urls to their correspoinding sections, so that we can use them with the tweets as media pieces

    Focus on details that are important for a comprehensive understanding of
    the blog.
 

Action 'Plan' don't exist, these are the only available Actions:
Tool Name: List files in directory
Tool Arguments: {'directory': {'description': 'Mandatory directory to list content', 'type': 'str'}}
Tool Description: A tool that can be used to recu

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/crewai/flow/flow.py", line 363, in _execute_single_listener
    listener_result = await self._execute_method(
  File "/usr/local/lib/python3.10/dist-packages/crewai/flow/flow.py", line 306, in _execute_method
    else method(*args, **kwargs)
  File "/tmp/ipykernel_1258/1661713420.py", line 58, in save_plan
    json.dump(plan.pydantic.model_dump(), f, indent=2)
AttributeError: 'str' object has no attribute 'pydantic'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/crewai/flow/flow.py", line 363, in _execute_single_listener
    listener_result = await self._execute_method(
  File "/usr/local/lib/python3.10/dist-packages/crewai/flow/flow.py", line 306, in _execute_method
    else method(*args, **kwargs)
  File "/tmp/ipykernel_1258/1661713420.py", line 69, in publish
    print(f"Here's the link to scheduled draft: {response['share_url']}")
TypeError: 'NoneType' object is no

#### Gpt-4o-mini

In [ ]:
response = flow.kickoff()

 Flow started with ID: 863b9022-7093-4afc-b7bf-0d584f383001
# fetching draft from: https://blog.dailydoseofds.com/p/evaluation-and-observability-for
# Planning content for: workdir/Evaluation and Observability for LLM Apps - by Avi Chawla.md
# Agent: Draft Analyzer
## Task: Analyze the markdown file at workdir/Evaluation and Observability for LLM Apps - by Avi Chawla.md to create a developer-focused
    technical overview

    1. Map out the core idea that the blog discusses

    2. Identify key sections and what each section is about

    3. For each section, extract all URLs that appear inside image markdown syntax ![](image_url)

    4. You must associate these identified image urls to their correspoinding sections, so that we can use them with the tweets as media pieces

    Focus on details that are important for a comprehensive understanding of
    the blog.


# Agent: Draft Analyzer
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"workdir/Evaluation and Ob

In [ ]:
response

In [ ]:
llm = LLM(
    model="ollama/llama3.2"
)

In [ ]:
import litellm
litellm.request_timeout=3600

## Define Agents and Tasks

In [ ]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
)

draft_analyzer = Agent(role ="Draft Analyzer",
                       goal="Analyze draft and identify key ideas, sections and technical concepts",
                       backstory=(""" You are a technical writer with years of experience writing, editing and
    reviewing technical blogs. You have a talent for understanding and
    documenting technical concepts."""),
                      tools=[DirectoryReadTool(),FileReadTool()],
                      verbose=True,
                      llm=llm)
#
twitter_thread_planner = Agent(role="Twitter Thread Planner",
                               goal="Create a Twitter thread plan based on the provided draft analysis",
                               backstory=(""" You are a technical writer with years of experience in converting long
    technical blogs into Twitter threads. You have a talent for breaking longform
    content into bite-sized tweets that are engaging and informative. And identify
    relevant urls to media that can be associated with a tweet."""),
                               tools=[DirectoryReadTool(),FileReadTool()],
                               verbose=True,
                               llm=llm)
#
# TASK
#
analyze_draft = Task(
  description=("""Analyze the markdown file at {draft_path} to create a developer-focused
    technical overview

    1. Map out the core idea that the blog discusses

    2. Identify key sections and what each section is about

    3. For each section, extract all URLs that appear inside image markdown syntax ![](image_url)

    4. You must associate these identified image urls to their correspoinding sections, so that we can use them with the tweets as media pieces

    Focus on details that are important for a comprehensive understanding of
    the blog."""),
  expected_output="""|
    A technical analysis containing:
    - Blog title and core concept/idea
    - Key technical sections identified with their main points
    - Important code examples or technical concepts covered
    - Key takeaways for developers
    - Relevant urls to media that are associated with the key sections and can be associated with a tweet, this must be done""",
  agent=draft_analyzer
)
create_twitter_thread_plan = Task(
  description=("""|
    Develop an engaging Twitter thread based on the draft analysis provided and closely follow the writing style prvided in the {path_to_example_threads}

    The thread should break down complex technical concepts into digestible, tweet-sized chunks
    that maintain technical accuracy while being accessible.

    Plan should include:
    - A strong hook tweet that captures attention, it should be under 10 words, it must be same as the title of the blog
    - Logical flow from basic to advanced concepts
    - Code snippets or key technical highlights that fit Twitter's format
    - Relevant urls to media that are associated with the key sections and must be associated with their corresponding tweets
    - Clear takeaways for engineering audience

    Make sure to cover:
    - close follow the writing style provided in the {path_to_example_threads}
    - The core problem being solved
    - Key technical innovations or approaches
    - Interesting implementation details
    - Real-world applications or benefits
    - Call to action for the conclusion
    - Add relevant urls to each tweet that can be associated with a tweet

    Focus on creating a narrative that technical audiences will find valuable
    while keeping each tweet concise, accessible and impactful."""),
  expected_output="""|
    A Twitter thread with a list of tweets, where each tweet has the following:
    - content
    - urls to media that are associated with the tweet, whenever possible
    - is_hook: true if the tweet is a hook tweet, false otherwise
""",
  agent=twitter_thread_planner,
  output_pydantic=Thread
)

planning_crew = Crew(
    agents=[draft_analyzer, twitter_thread_planner],
    tasks=[analyze_draft, create_twitter_thread_plan],
    verbose=False
)

LLM value is already an LLM object
LLM value is already an LLM object


In [ ]:
linkedin_post_planner = Agent(role="LinkedIn Post Planner",
                              goal="Create an engaging LinkedIn post based on the provided draft analysis",
                              backstory=("""You are a technical writer with extensive experience crafting technical LinkedIn
    content. You excel at distilling technical concepts into clear, authoritative posts
    that resonate with a professional audience while maintaining technical accuracy.
    You know how to balance technical depth with accessibility and incorporate relevant
    hashtags and mentions to maximize engagement."""),
                              verbose=True,
                              llm=llm)

create_linkedin_post_plan = Task(
  description=("""|
    Develop a comprehensive LinkedIn post based on the draft analysis provided

    The post should present technical content in a professional, long-form format
    while maintaining engagement and readability.

    Plan should include:
    - An attention-grabbing opening statement, it should be same as the title of the blog
    - Well-structured body that breaks down the technical content
    - Professional tone suitable for LinkedIn's business audience
    - One main blog URL placed strategically at the end of the post
    - Strategic use of line breaks and formatting
    - Relevant hashtags (3-5 maximum)

    Make sure to cover:
    - The core technical problem and its business impact
    - Key solutions and technical approaches
    - Real-world applications and benefits
    - Professional insights or lessons learned
    - Clear call to action

    Focus on creating content that resonates with both technical professionals
    and business leaders while maintaining technical accuracy."""),
  expected_output="""|
    A LinkedIn post plan containing:
      - content
      - a main blog url that is associated with the post""",
  agent=linkedin_post_planner,
  output_pydantic=LinkedInPost
)

linkedin_planning_crew = Crew(
    agents=[draft_analyzer, linkedin_post_planner],
    tasks=[analyze_draft, create_linkedin_post_plan],
    verbose=False
)

LLM value is already an LLM object


In [ ]:
post_type = "linkedin"
#blog_post_url = "https://blog.dailydoseofds.com/p/5-agentic-ai-design-patterns"
blog_post_url = "https://blog.dailydoseofds.com/p/build-human-like-memory-for-your"

In [ ]:
from crewai.flow.flow import Flow, listen, start, router, or_
from langchain_core.output_parsers import JsonOutputParser
parser = JsonOutputParser()

class ContentPlanningState(BaseModel):
  """
  State for the content planning flow
  """
  blog_post_url: str = blog_post_url
  draft_path: Path = "workdir/"
  post_type: str = "twitter"
  path_to_example_threads: str = "workdir/example_threads.txt"

class CreateContentPlanningFlow(Flow[ContentPlanningState]):
  # Scrape the blog post
  # No need for AI Agents on this step, so we just use regular Python code
  @start()
  def scrape_blog_post(self):
    print(f"# fetching draft from: {self.state.blog_post_url}")
    app = FirecrawlApp(api_key=os.getenv("FIRECRAWL_API_KEY"))
    scrape_result = app.scrape_url(self.state.blog_post_url, params={'formats': ['markdown', 'html']})
    try:
      title = scrape_result['metadata']['title']
    except Exception as e:
      title = str(uuid.uuid4())
    self.state.draft_path = f'workdir/{title}.md'
    with open(self.state.draft_path, 'w') as f:
      f.write(scrape_result['markdown'])
    return self.state

  @router(scrape_blog_post)
  def select_platform(self):
    if self.state.post_type == "twitter":
      return "twitter"
    elif self.state.post_type == "linkedin":
      return "linkedin"

  @listen("twitter")
  def twitter_draft(self):
    print(f"# Planning content for: {self.state.draft_path}")
    result = planning_crew.kickoff(inputs={'draft_path': self.state.draft_path, 'path_to_example_threads': self.state.path_to_example_threads})
    print(f"# Planned content for {self.state.draft_path}:")
    for tweet in result.pydantic.tweets:
        print(f"    - {tweet.content}")
    return result

  @listen("linkedin")
  def linkedin_draft(self):
    print(f"# Planning content for: {self.state.draft_path}")
    result = linkedin_planning_crew.kickoff(inputs={'draft_path': self.state.draft_path})
    print(f"# Planned content for {self.state.draft_path}:")
    print(f"Response from Linkein Content    - {result.pydantic.content}")
    new_result = parser.parse(result.raw)
    print(f"new_result:{new_result}")
    return result

  @listen(or_(twitter_draft, linkedin_draft))
  def save_plan(self, plan):
    with open(f'thread/{self.state.draft_path.split("/")[-1]}_{self.state.post_type}.json', 'w') as f:
        json.dump(plan.pydantic.model_dump(), f, indent=2)

  @listen(or_(twitter_draft, linkedin_draft))
  def publish(self, plan):
    print(f"# Publishing thread for: {self.state.draft_path}")
    ## Schedule for 1 hour from now
    response = schedule(
        thread_model=plan,
        post_type=self.state.post_type
    )
    print(f"# Thread scheduled for: {self.state.draft_path}")
    print(f"Here's the link to scheduled draft: {response['share_url']}")

In [ ]:

flow = CreateContentPlanningFlow()
flow.state.post_type = post_type
flow.state

ContentPlanningState(blog_post_url='https://blog.dailydoseofds.com/p/build-human-like-memory-for-your', draft_path='workdir/', post_type='linkedin', path_to_example_threads='workdir/example_threads.txt')

In [ ]:
response = flow.kickoff()

# fetching draft from: https://blog.dailydoseofds.com/p/build-human-like-memory-for-your
# Planning content for: workdir/Build Human-like Memory for Your AI Agents - by Avi Chawla.md
# Agent: Draft Analyzer
## Task: Analyze the markdown file at workdir/Build Human-like Memory for Your AI Agents - by Avi Chawla.md to create a developer-focused
    technical overview

    1. Map out the core idea that the blog discusses

    2. Identify key sections and what each section is about

    3. For each section, extract all URLs that appear inside image markdown syntax ![](image_url)

    4. You must associate these identified image urls to their correspoinding sections, so that we can use them with the tweets as media pieces

    Focus on details that are important for a comprehensive understanding of
    the blog.
 

Action 'Read files in directory' don't exist, these are the only available Actions:
Tool Name: List files in directory
Tool Arguments: {'directory': {'description': 'Mandatory di